In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import yfinance as yf

start = datetime.datetime(1998, 4, 1)
end = datetime.datetime(2025, 3, 31)


# -------------------------------
# 1. Retrieve S&P 500 Data (Daily)
# -------------------------------
# The FRED series "SP500" provides daily data.
sp500_raw = yf.download('^GSPC', start=start, end=end)
sp500_daily = sp500_raw[['Close']].rename(columns={'Close': 'SP500'})
# Resample to monthly data by taking the last available closing value of each month
sp500_monthly = sp500_daily.resample('ME').last()
# sp500_monthly = sp500_monthly.reset_index()
# sp500_monthly.columns = ['Date', 'SP500']

# -------------------------------
# 2. Retrieve Bond and Interest Rate Data
# -------------------------------
# 10-Year Treasury Constant Maturity Rate (GS10) - monthly averages
gs10 = web.DataReader('GS10', 'fred', start, end)
gs10_monthly = gs10.resample('ME').last()

# Effective Federal Funds Rate (FEDFUNDS) - provided as monthly averages
fedfunds = web.DataReader('FEDFUNDS', 'fred', start, end)
fedfunds_monthly = fedfunds.resample('ME').last()

# -------------------------------
# 3. Retrieve CPI Data and Calculate Inflation
# -------------------------------
# CPI for All Urban Consumers (CPIAUCSL) - seasonally adjusted monthly data
cpi = web.DataReader('CPIAUCSL', 'fred', start, end)
cpi_monthly = cpi.resample('ME').last()

# Convert CPI to annual inflation rate (year-over-year percentage change)
cpi_inflation = cpi_monthly.pct_change(12) * 100

# -------------------------------
# 4. Combine All Data into a Single DataFrame
# -------------------------------
# Concatenate the series along columns
data = pd.concat([ sp500_monthly,gs10_monthly,fedfunds_monthly,cpi_inflation], axis=1)

# Rename columns for clarity
data.columns = ['SP500', 'GS10', 'FEDFUNDS', 'Inflation']
data.to_csv('./collected_data.csv')

# Total number of rows in the DataFrame
total_rows = len(data)

# Count the missing values per column
missing_counts = data.isna().sum()

# Count the non-missing values per column (optional)
non_missing_counts = data.count()

# Calculate the percentage of missing values per column
missing_percentage = (missing_counts / total_rows) * 100

# Create a summary DataFrame
summary = pd.DataFrame({
    'Total Rows': total_rows,
    'Non-Missing': non_missing_counts,
    'Missing': missing_counts,
    'Missing (%)': missing_percentage
})

print(summary)

print('========================================')

# Extract rows with any NaN values
missing_data = data[data.isna().any(axis=1)]
print("Rows with missing data:")
print(missing_data)
print('========================================')

for date, row in missing_data.iterrows():
    missing_cols = row[row.isna()].index.tolist()
    print(f"Month end {date.strftime('%Y-%m-%d')} is missing data for: {missing_cols}")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


           Total Rows  Non-Missing  Missing  Missing (%)
SP500             324          324        0     0.000000
GS10              324          324        0     0.000000
FEDFUNDS          324          324        0     0.000000
Inflation         324          312       12     3.703704
Rows with missing data:
                  SP500  GS10  FEDFUNDS  Inflation
1998-04-30  1111.750000  5.64      5.45        NaN
1998-05-31  1090.819946  5.65      5.49        NaN
1998-06-30  1133.839966  5.50      5.56        NaN
1998-07-31  1120.670044  5.46      5.54        NaN
1998-08-31   957.280029  5.34      5.55        NaN
1998-09-30  1017.010010  4.81      5.51        NaN
1998-10-31  1098.670044  4.53      5.07        NaN
1998-11-30  1163.630005  4.83      4.83        NaN
1998-12-31  1229.229980  4.65      4.68        NaN
1999-01-31  1279.640015  4.72      4.63        NaN
1999-02-28  1238.329956  5.00      4.76        NaN
1999-03-31  1286.369995  5.23      4.81        NaN
Month end 1998-04-30 is miss

In [2]:
data

,SP500,GS10,FEDFUNDS,Inflation
1998-04-30,1111.750000,5.64,5.45,NaN
1998-05-31,1090.819946,5.65,5.49,NaN
1998-06-30,1133.839966,5.50,5.56,NaN
1998-07-31,1120.670044,5.46,5.54,NaN
1998-08-31,957.280029,5.34,5.55,NaN
...,...,...,...,...
2024-11-30,6032.379883,4.36,4.64,2.714168
2024-12-31,5881.629883,4.39,4.48,2.872366
2025-01-31,6040.529785,4.63,4.33,2.999413
2025-02-28,5954.500000,4.45,4.33,2.814270


In [3]:
cpi_inflation

,CPIAUCSL
DATE,
1998-04-30,NaN
1998-05-31,NaN
1998-06-30,NaN
1998-07-31,NaN
1998-08-31,NaN
...,...
2024-11-30,2.714168
2024-12-31,2.872366
2025-01-31,2.999413


In [4]:
data

,SP500,GS10,FEDFUNDS,Inflation
1998-04-30,1111.750000,5.64,5.45,NaN
1998-05-31,1090.819946,5.65,5.49,NaN
1998-06-30,1133.839966,5.50,5.56,NaN
1998-07-31,1120.670044,5.46,5.54,NaN
1998-08-31,957.280029,5.34,5.55,NaN
...,...,...,...,...
2024-11-30,6032.379883,4.36,4.64,2.714168
2024-12-31,5881.629883,4.39,4.48,2.872366
2025-01-31,6040.529785,4.63,4.33,2.999413
2025-02-28,5954.500000,4.45,4.33,2.814270
